## Weight  of evidence

Weight of Evidence (WoE) was developed primarily for the credit and financial industries to help build more predictive models to evaluate the risk of loan default. That is, to predict how likely the money lent to a person or institution is to be lost. Thus, Weight of Evidence is a measure of the "strength” of a grouping technique to separate good and bad risk (default). 

- WoE will be 0 if the P(Goods) / P(Bads) = 1, that is, if the outcome is random for that group.
- If P(Bads) > P(Goods) the odds ratio will be < 1 and,
- WoE will be < 0 if,  P(Goods) > P(Bads).

WoE is well suited for Logistic Regression, because the Logit transformation is simply the log of the odds, i.e., ln(P(Goods)/P(Bads)). Therefore, by using WoE-coded predictors in logistic regression, the predictors are all prepared and coded to the same scale, and the parameters in the linear logistic regression equation can be directly compared.

The WoE transformation has three advantages:

- It creates a monotonic relationship between the target and the independent variables.
- It orders the categories on a "logistic" scale which is natural for logistic regression
- The transformed variables can then be compared because they are on the same scale. Therefore, it is possible to determine which one is more predictive.

The WoE also has a limitation:

- Prone to cause over-fitting


For more details follow this [article](https://www.listendata.com/2015/03/weight-of-evidence-woe-and-information.html#Steps-of-Calculating-WOE) and this [article](https://blogs.oracle.com/r/computing-weight-of-evidence-woe-and-information-value-iv).


## In this demo:

We will see how to perform one hot encoding with Category encoders using the Titanic dataset.

For guidelines to obtain the dataset, visit **section 2** of the course.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

from category_encoders.woe import WOEEncoder

In [2]:
# load dataset

data = pd.read_csv(
    "../../titanic.csv", usecols=["cabin", "sex", "embarked", "survived"]
)

data.head()

survived     sex cabin embarked
0         1  female    B5        S
1         1    male   C22        S
2         0  female   C22        S
3         0    male   C22        S
4         0  female   C22        S

In [3]:
# let's remove observations with na in embarked

data.dropna(subset=["embarked"], inplace=True)
data.shape

(1307, 4)

In [4]:
# Now we extract the first letter of the cabin
# to create a simpler variable for the demo

data["cabin"] = data["cabin"].astype(str).str[0]

In [5]:
# and we remove the observations where cabin = T
# because only few observations show T in the dataset

data = data[data["cabin"] != "T"]
data.shape

(1306, 4)

## Weight of Evidence with Category Encoders

If using Category encpders, instead of pandas, we do not need to keep the target variable in the training dataset.

In [6]:
# let's separate into training and testing set

X_train, X_test, y_train, y_test = train_test_split(
    data[["cabin", "sex", "embarked"]],  # predictors
    data["survived"],  # target
    test_size=0.3,  # percentage of obs in test set
    random_state=0,
)  # seed to ensure reproducibility

X_train.shape, X_test.shape

((914, 3), (392, 3))

In [7]:
woe_enc = WOEEncoder(cols=["cabin", "sex", "embarked"])

In [8]:
woe_enc.fit(X_train, y_train)

WOEEncoder(cols=['cabin', 'sex', 'embarked'])

In [9]:
# the mappings in category encoders are not so easy to interpret

woe_enc.mapping

{'cabin': cabin
  1   -0.374548
  2    1.287500
  3    1.295799
  4    0.901246
  5    1.518942
  6    0.188888
  7    1.488171
  8    1.200488
 -1    0.000000
 -2    0.000000
 dtype: float64,
 'sex': sex
  1   -1.008094
  2    1.525265
 -1    0.000000
 -2    0.000000
 dtype: float64,
 'embarked': embarked
  1   -0.164851
  2    0.654977
  3   -0.307696
 -1    0.000000
 -2    0.000000
 dtype: float64}

In [10]:
X_train = woe_enc.transform(X_train)
X_test = woe_enc.transform(X_test)

# let's explore the result
X_train.head()

cabin       sex  embarked
843  -0.374548 -1.008094 -0.164851
869  -0.374548 -1.008094 -0.164851
430  -0.374548  1.525265 -0.164851
481  -0.374548  1.525265  0.654977
1308 -0.374548 -1.008094 -0.164851